In [1]:
import pandas as pd
import pykrige as krig
import numpy as np
import matplotlib.pyplot as plt
from sklearn. metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error
import pickle

In [2]:

pd.set_option('display.max_columns', None)
df_original = pd.read_csv('data/Data.csv')
df = df_original.copy()
df = df[df['FINO']!='-']
df = df[df['STYPE']!='RC']
df['FINO'] = df['FINO'].astype(float)


df.drop(['HG', 'AS', 'HUMEDAD', 'PH', 'P80', 'S', 'C', 'AUCN',
 'FROM', 'TO', 'LITHO','DOM','LENGTH'],axis=1, inplace=True)
df

/tmp/ipykernel_48908/2103183600.py:2: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv('data/Data.csv')


,X,Y,Z,BHID,AZONE,YEAR,AU,AG,FINO,CU,STYPE
735,414931.1150,8029585.713,4438.736000,103051731,MORR,2017,0.382,1.3,9.89,0.0023,BH
739,414935.6830,8029597.151,4438.691000,103051737,MORR,2017,0.806,1.2,21.59,0.002,BH
755,414904.3220,8029596.428,4438.617000,103051777,MORR,2017,0.467,0.6,31.86,0.0027,BH
757,414916.0930,8029605.682,4438.433000,103051781,MORR,2017,0.898,0.9,11.42,0.0028,BH
766,414901.4260,8029618.044,4438.617000,103051805,MORR,2017,0.576,0.7,19.86,0.0019,BH
...,...,...,...,...,...,...,...,...,...,...,...
43942,414610.8067,8029522.426,4320.951727,PR-PU18-09,SIGR,2018,0.66769646,24.67268128,12.00,0.0052597,DD
43943,414613.6406,8029518.451,4315.374362,PR-PU18-09,SIGR,2018,0.36839966,7.50657673,12.00,0.00523288,DD
43944,414616.4650,8029514.469,4309.796385,PR-PU18-09,SIGR,2018,0.44208263,6.41652614,12.00,0.00876105,DD
43945,414619.2743,8029510.485,4304.212379,PR-PU18-09,SIGR,2018,0.44088364,3.30708263,12.00,0.00121619,DD


In [4]:
# UK = krig.OrdinaryKriging3D(
#     x=df['X'],
#     y=df['Y'],
#     z=df['Z'],
#     val=df['FINO'],
#     variogram_model="linear"
# )

# # Open a file and use dump()
# with open('gaussian_kriggin.pkl', 'wb') as file:
      
#     # A new file will be created
#     pickle.dump(UK, file)

In [3]:


with open('gaussian_kriggin.pkl', 'rb') as file:
      
    # Call load method to deserialze
    UK = pickle.load(file)



In [11]:
df_sample = df.sample(10)
gridx = np.arange((df['X'].min()), (df['X'].max()), 50)
gridy = np.arange((df['Y'].min()), (df['Y'].max()), 50)
gridz = np.arange((df['Z'].min()), (df['Z'].max()), 10)
z, ss = UK.execute(
    style="grid",
    xpoints= df_sample['X'],
    ypoints= df_sample['Y'],
    zpoints= df_sample['Z'])

In [15]:
df_result = pd.DataFrame(columns=['x','y','z','finos'])
for i in range(len(gridz)):
    for j in range(len(gridy)):
        for k in range(len(gridx)):
            df_temp = pd.DataFrame({
                'x': [gridx[k]],
                'y': [gridy[j]],
                'z': [gridz[i]],
                'finos': [z.data[i][j][k]]
            })
            df_result = pd.concat([df_result,df_temp])
df_result.reset_index(inplace=True, drop=True)
df_result

TypeError: object of type 'int' has no len()

In [14]:
z.shape

(10, 10, 10)

In [6]:
print("r2: ", r2_score(df_sample['FINO'], df_result['finos']))
print("mae: ", mean_absolute_error(df['FINO'], df_result['finos']))
print("mape: ", mean_absolute_percentage_error(df['FINO'], df_result['finos']))

ValueError: Found input variables with inconsistent numbers of samples: [6610, 10672]